In [1]:
!pip install datasets


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset
import os
import random
import pandas as pd
import json
import numpy as np
from collections import Counter

### Carico e filtro il dataset Personas

In [3]:
dataset = load_dataset("argilla/FinePersonas-v0.1", split="train")
dataset[0]

{'id': '<urn:uuid:a0f6a31d-26d9-465a-85af-fcef7ca88935>',
 'persona': 'A professional R programmer or researcher, likely a data analyst or statistician, familiar with the intricacies of the R language and its debugging tools.',
 'labels': '["Data Analysis", "Statistical Expertise", "R Programming"]'}

In [4]:
len(dataset)

21071228

In [5]:
# Prendi solo i primi 10.000 
subset = dataset.select(range(10000))
len(subset)

10000

In [6]:
df_personas = subset.to_pandas()

In [7]:
# Visualizza le colonne del DataFrame
df_personas.columns

# Converte la stringa in lista
df_personas["parsed_labels"] = df_personas["labels"].apply(json.loads)

# Crea un set con tutte le label uniche
unique_labels = set(label for sublist in df_personas["parsed_labels"] for label in sublist)

# Ordinale alfabeticamente
unique_labels = sorted(unique_labels)

# Mostrale
for label in unique_labels:
    print(label)

print(len(unique_labels))

ADHD
Academia
Academia/Research
Academic
Academic Editor
Academics
Accessibility Specialist
Adventure
Aerospace
Agriculture
Allergy Specialist
Alternative Beliefs
Anatomy
Animal Behavior
Animal Care Specialist
Animal Health
Anthropology
Anti-Evolutionism
Aquarium Enthusiast
Aquatic Life Specialist
Arachnology
Archaeology
Architecture
Art
Art Historian
Art Professional
Art/Design
Artificial Intelligence
Arts
Arts and Crafts
Astronomy
Audio
Audiology Related
Automation Specialist
Avian Expert
Aviation
Biological Science
Biologist
Biology
Biomedical Research
Birds of Prey
Business
CAD User
Cartography
Child Development
Christian Apologetics
Climate Change
Climate Change Researcher
Cognitive Science
Collector
Common_Core_State_Standards
Communication
Communications
Computer Networking
Computer Science
Conservation
Conspiracy Theory
Construction
Content Curator
Craftsmanship
Creationism
Criminal Investigation
Culinary Professional
Cultural Specialist
Culture
Cybersecurity
DIY Hobbyist
Dance

In [8]:
# Conta le occorrenze per ogni specifico campo
all_labels = [label for sublist in df_personas["parsed_labels"] for label in sublist]
label_counts = Counter(all_labels)

# Ordina per frequenza e stampa
for label, count in label_counts.most_common():
    print(f"{label}: {count}")

Academia: 3830
Education: 3697
Specialized Expertise: 2622
Scientific: 1859
Environmental: 1731
Educator: 1452
Professional: 1315
Medical Professional: 1314
Healthcare Specialist: 1234
Science: 1102
Space Exploration: 718
N: 454
o: 454
n: 454
e: 454
Academic: 432
Conservation: 406
Teaching: 329
Healthcare: 245
Educational: 214
Programmer: 209
Technologist: 209
Healthcare Professional: 185
Engineering: 180
Technology: 174
Geology: 173
Earth Sciences: 165
Physics: 151
History: 146
Religion: 141
Researcher: 138
Neuroscience: 125
Paleontology: 120
Psychology: 113
Sustainability: 113
Environmentalism: 103
Child Development: 98
Mental Health: 88
Music: 88
Research: 81
Anthropology: 76
Archaeology: 76
Wildlife: 75
Health Educator: 64
Medical: 58
Scientist: 57
Computer Networking: 57
Cybersecurity: 57
Construction: 56
Electronics: 55
Economics: 52
Finance: 52
Medical Specialist: 52
Architecture: 52
Creationism: 51
Anti-Evolutionism: 51
Christian Apologetics: 51
Space: 50
Public Health: 50
Phys

In [9]:
import random

# Lista delle etichette selezionate
selected_labels = ["Healthcare", "Technology", "Religion", "Music"]
profiles_by_label = {label: [] for label in selected_labels}

# Filtriamo il DataFrame per ogni etichetta
for label in selected_labels:
    filtered_df = df_personas[df_personas['labels'].apply(lambda x: label in x)]
    profiles_by_label[label] = filtered_df

# Estrazione profili in modo randomico, bilanciando il numero di profili (totale di 143)
total_profiles = 143
num_profiles_per_label = total_profiles // len(selected_labels)

selected_profiles = []
for label in selected_labels:
    sampled_profiles = random.sample(list(profiles_by_label[label].iterrows()), num_profiles_per_label)
    selected_profiles.extend(sampled_profiles)

remaining_profiles = total_profiles - len(selected_profiles)
if remaining_profiles > 0:
    extra_samples = random.sample(list(profiles_by_label[selected_labels[0]].iterrows()), remaining_profiles)
    selected_profiles.extend(extra_samples)

# Conteggio delle etichette selezionate
selected_label_counts = {label: 0 for label in selected_labels}
for _, row in selected_profiles:
    for label in selected_labels:
        if label in row['labels']:
            selected_label_counts[label] += 1

# Stampa il conteggio delle etichette
print("Conteggio delle etichette selezionate:")
for label, count in selected_label_counts.items():
    print(f"{label}: {count}")


Conteggio delle etichette selezionate:
Healthcare: 38
Technology: 35
Religion: 35
Music: 35


In [10]:
# Verifica se ci sono agenti con più di una delle etichette selezionate
agents_with_multiple_labels = []

for _, row in selected_profiles:
    # Conta quante etichette selezionate ha questo agente
    count = sum([1 for label in selected_labels if label in row['labels']])
    if count > 1:
        agents_with_multiple_labels.append(row)

# Stampa il numero di agenti con più di una etichetta selezionata
print(f"Numero di agenti con più di una delle etichette selezionate: {len(agents_with_multiple_labels)}")


Numero di agenti con più di una delle etichette selezionate: 0


In [11]:
# Lista per raccogliere i dati
data = []

# Per ogni agente selezionato, prendi la personalità e il label selezionato
for _, row in selected_profiles:
    # Trova il label selezionato
    for label in selected_labels:
        if label in row['labels']:
            data.append([row['persona'], label])  # Aggiungi la personalità e il label
            break  # Un agente ha solo un label tra quelli selezionati

# Crea un DataFrame con le due colonne
df_labels = pd.DataFrame(data, columns=["Personality", "Label"])

#check labels
counter1=0
counter2=0
counter3=0
counter4=0


for _, i in df_labels.iterrows():
    if i["Label"] == "Healthcare":
        counter1 += 1
    elif i["Label"] == "Technology":
        counter2 += 1
    elif i["Label"] == "Religion":
        counter3 += 1
    elif i["Label"] == "Music":
        counter4 += 1


print("Healthcare: ", counter1)
print("Technology: ", counter2)
print("Religion: ", counter3)
print("Music: ", counter4)

df_labels

Healthcare:  38
Technology:  35
Religion:  35
Music:  35


,Personality,Label
0,"A researcher in neurodevelopmental disorders, ...",Healthcare
1,An immunologist or a vaccine researcher with a...,Healthcare
2,"A healthcare professional, likely a cardiologi...",Healthcare
3,A registered nurse or a medical professional i...,Healthcare
4,"A health journalist, possibly specializing in ...",Healthcare
...,...,...
138,A musicologist or a classical music historian ...,Music
139,A classical music historian with an emphasis o...,Music
140,A primary school teacher or a pediatric nutrit...,Healthcare
141,A microbiologist specializing in the study of ...,Healthcare


In [18]:
# Limita "Healthcare" a 35 occorrenze
df_limited = pd.concat([
    df_labels[df_labels["Label"] == "Healthcare"].head(35),
    df_labels[df_labels["Label"] == "Technology"],
    df_labels[df_labels["Label"] == "Religion"],
    df_labels[df_labels["Label"] == "Music"]
], ignore_index=True)

# Verifica che i conteggi siano corretti
print("Healthcare:", (df_limited["Label"] == "Healthcare").sum())
print("Technology:", (df_limited["Label"] == "Technology").sum())
print("Religion:", (df_limited["Label"] == "Religion").sum())
print("Music:", (df_limited["Label"] == "Music").sum())

# Visualizza il nuovo DataFrame
df_limited


Healthcare: 35
Technology: 35
Religion: 35
Music: 35


,Personality,Label
0,"A researcher in neurodevelopmental disorders, ...",Healthcare
1,An immunologist or a vaccine researcher with a...,Healthcare
2,"A healthcare professional, likely a cardiologi...",Healthcare
3,A registered nurse or a medical professional i...,Healthcare
4,"A health journalist, possibly specializing in ...",Healthcare
...,...,...
135,A music theory professor specializing in class...,Music
136,A musicologist with a focus on historical keyb...,Music
137,"A flute instructor or private flute teacher, p...",Music
138,A musicologist or a classical music historian ...,Music


In [19]:
for _, i in df_labels.iterrows():
    if i["Label"] == "Healthcare":
       print(i["Personality"])

A researcher in neurodevelopmental disorders, specifically focused on the genetic and biological aspects of autism.
An immunologist or a vaccine researcher with a focus on understanding the molecular mechanisms of vaccine adjuvants and the innate immune system.
A healthcare professional, likely a cardiologist or primary care physician, who focuses on educating and guiding patients with hypertension on monitoring and managing their blood pressure at home.
A registered nurse or a medical professional in critical care or emergency medicine focused on fluid management and patient hydration.
A health journalist, possibly specializing in infectious diseases and microbiome research, with a strong background in science and medicine, and a keen interest in exploring the impact of human activity on public health and the environment.
A vascular surgeon or a cardiologist focused on abdominal aortic aneurysms (AAAs), likely in an educational or informative role, or possibly a medical student or int

### Profili di attività

In [15]:
SO = "You are a Silent Observer. Do not post, share, like, dislike, or comment under any normal circumstance. Your only job is to watch and absorb without leaving any trace. Remain invisible in the conversation. Avoid all posting, sharing, or reacting unless there is a strong external trigger."

OS = "You are an Occasional Sharer. Stay mostly silent and inactive. Only retweet content that clearly aligns with your interests; refrain from liking, disliking, commenting, or posting original content."

OE = "You are an Occasional Engager. Your engagement is limited to minimal reactions. Occasionally like, dislike, or comment on posts that catch your eye. Do not retweet or write original tweets."

BP = "You are a Balanced Participant. In each cycle, mix original tweets, retweets, and occasional likes or dislikes, or comments. Maintain a steady, balanced level of engagement. Contribute regularly with your own posts, amplify others via retweets, and use reactions or comments when appropriate."

CA = "You are a Content Amplifier. Spend most of your time sharing others’ posts and reacting to them with likes, dislikes or comments. Posting new content is secondary. Your main task is to retweet frequently and support others with likes, dislikes or comments."

PC = "You are a Proactive Contributor. Lead the conversation with your own original tweets. You occasionally engage with others through comments, likes, or dislikes, but you tend to avoid retweets. Prioritize expressing your own ideas and perspectives."

IE = "You are an Interactive Enthusiast. Your primary mode of participation is reactive: comment extensively, like or dislike content regularly. Posting or sharing happens only occasionally. Engage deeply through comments, likes, and dislikes."

activity_profiles = {
    "SO": SO,
    "OS": OS,
    "OE": OE,
    "BP": BP,
    "CA": CA,
    "PC": PC,
    "IE": IE,
}

### Generazione 1000 (personalità + profilo attività)

In [72]:
#CODICE PER GENERARE CIRCA 1000 AGENTI (SIMULAZIONE GRANDE)

# Lista per raccogliere i dati finali
final_data = []

# Contatore globale degli agenti
agent_counter = 1

# Per ogni personalità e label
for personality, label in df_labels.values:
    for code, profile in activity_profiles.items():
        agent_name = f"Agent{agent_counter}_{label}_{code}"
        description = f"You are {personality}\n{profile}"
        final_data.append([agent_name, description])
        agent_counter += 1

# Creazione del DataFrame finale
df_complete_personality = pd.DataFrame(final_data, columns=["Agent_Name", "Description"])

# Visualizza le prime righe
df_complete_personality


,Agent_Name,Description
0,Agent1_Healthcare_SO,You are A genetic counselor or a medical profe...
1,Agent2_Healthcare_OS,You are A genetic counselor or a medical profe...
2,Agent3_Healthcare_OE,You are A genetic counselor or a medical profe...
3,Agent4_Healthcare_BP,You are A genetic counselor or a medical profe...
4,Agent5_Healthcare_CA,You are A genetic counselor or a medical profe...
...,...,...
996,Agent997_Healthcare_OE,You are A community health educator or public ...
997,Agent998_Healthcare_BP,You are A community health educator or public ...
998,Agent999_Healthcare_CA,You are A community health educator or public ...
999,Agent1000_Healthcare_PC,You are A community health educator or public ...


In [22]:
# Lista per raccogliere i dati finali
final_data = []

# Contatore globale degli agenti
agent_counter = 1

# Per ogni personalità e label
for personality, label in df_limited.values:
    for code, profile in activity_profiles.items():
        # Nome agente minuscolo con codice comportamentale
        agent_name = f"agent{agent_counter}_{label}_{code}".lower()
        # Descrizione contenente solo la personalità
        description = f"You are {personality}"
        final_data.append([agent_name, description])
        agent_counter += 1

# Creazione del DataFrame finale
df_complete_personality = pd.DataFrame(final_data, columns=["Agent_Name", "Description"])

# Visualizza le prime righe
df_complete_personality

,Agent_Name,Description
0,agent1_healthcare_so,You are A researcher in neurodevelopmental dis...
1,agent2_healthcare_os,You are A researcher in neurodevelopmental dis...
2,agent3_healthcare_oe,You are A researcher in neurodevelopmental dis...
3,agent4_healthcare_bp,You are A researcher in neurodevelopmental dis...
4,agent5_healthcare_ca,You are A researcher in neurodevelopmental dis...
...,...,...
975,agent976_music_oe,You are A classical music historian with an em...
976,agent977_music_bp,You are A classical music historian with an em...
977,agent978_music_ca,You are A classical music historian with an em...
978,agent979_music_pc,You are A classical music historian with an em...


In [23]:
# Nome della cartella da creare
folder_name = "personas_980"

# Crea la cartella se non esiste già
os.makedirs(folder_name, exist_ok=True)

# Per ogni riga del DataFrame, crea un file .txt
for _, row in df_complete_personality.iterrows():
    agent_filename = f"{row['Agent_Name']}.txt"
    filepath = os.path.join(folder_name, agent_filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(row['Description'])

In [82]:
print(df_complete_personality.iloc[475]["Description"])

You are A physics or materials science researcher with an interest in innovative applications of electromagnetic principles, likely focused on developing novel technologies such as maglev systems.
You are an Interactive Enthusiast. Your primary mode of participation is reactive: comment extensively, like or dislike content regularly. Posting or sharing happens only occasionally. Engage deeply through comments, likes, and dislikes.


In [83]:
df_complete_personality.to_csv("Healthcare_Technology_Religion_Music_Personas.csv", index=False)

In [3]:
df_complete_personality_1000 = pd.read_csv("Healthcare_Technology_Religion_Music_Personas.csv")
df_complete_personality_1000

,Agent_Name,Description
0,Agent1_Healthcare_SO,You are A genetic counselor or a medical profe...
1,Agent2_Healthcare_OS,You are A genetic counselor or a medical profe...
2,Agent3_Healthcare_OE,You are A genetic counselor or a medical profe...
3,Agent4_Healthcare_BP,You are A genetic counselor or a medical profe...
4,Agent5_Healthcare_CA,You are A genetic counselor or a medical profe...
...,...,...
996,Agent997_Healthcare_OE,You are A community health educator or public ...
997,Agent998_Healthcare_BP,You are A community health educator or public ...
998,Agent999_Healthcare_CA,You are A community health educator or public ...
999,Agent1000_Healthcare_PC,You are A community health educator or public ...


In [4]:
# Crea la cartella (se non esiste già)
output_dir = "personas_1000"
os.makedirs(output_dir, exist_ok=True)

# Itera attraverso ogni riga del DataFrame
for index, row in df_complete_personality_1000.iterrows():
    agent_name = row['Agent_Name']
    description = row['Description']
    
    # Crea il percorso del file
    file_path = os.path.join(output_dir, f"{agent_name}.txt")
    
    # Scrivi la descrizione nel file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(description)

print(f"Creati {len(df_complete_personality_1000)} file nella cartella '{output_dir}'")

Creati 1001 file nella cartella 'personas_1000'


In [3]:
#imposto tutti i nomi dei file in minuscolo
# Percorso della cartella da modificare
folder_path = "personas_1000"

# Controlla se la cartella esiste
if not os.path.exists(folder_path):
    print(f"La cartella '{folder_path}' non esiste.")

# Cicla su tutti i file nella cartella
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        lowercase_name = filename.lower()
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, lowercase_name)
        
        # Evita di rinominare se il nome è già in minuscolo
        if old_path != new_path:
            os.rename(old_path, new_path)
            print(f"Rinominato: {filename} → {lowercase_name}")

Rinominato: Agent1000_Healthcare_PC.txt → agent1000_healthcare_pc.txt
Rinominato: Agent1001_Healthcare_IE.txt → agent1001_healthcare_ie.txt
Rinominato: Agent100_Healthcare_OS.txt → agent100_healthcare_os.txt
Rinominato: Agent101_Healthcare_OE.txt → agent101_healthcare_oe.txt
Rinominato: Agent102_Healthcare_BP.txt → agent102_healthcare_bp.txt
Rinominato: Agent103_Healthcare_CA.txt → agent103_healthcare_ca.txt
Rinominato: Agent104_Healthcare_PC.txt → agent104_healthcare_pc.txt
Rinominato: Agent105_Healthcare_IE.txt → agent105_healthcare_ie.txt
Rinominato: Agent106_Healthcare_SO.txt → agent106_healthcare_so.txt
Rinominato: Agent107_Healthcare_OS.txt → agent107_healthcare_os.txt
Rinominato: Agent108_Healthcare_OE.txt → agent108_healthcare_oe.txt
Rinominato: Agent109_Healthcare_BP.txt → agent109_healthcare_bp.txt
Rinominato: Agent10_Healthcare_OE.txt → agent10_healthcare_oe.txt
Rinominato: Agent110_Healthcare_CA.txt → agent110_healthcare_ca.txt
Rinominato: Agent111_Healthcare_PC.txt → agent

### Generazione 70 (personalità + profilo attività)


In [13]:
# Fissa il numero di personalità per label
label_counts = {
    "Healthcare": 3,
    "Technology": 3,
    "Religion": 2,
    "Music": 2
}

selected_profiles = []
for label, count in label_counts.items():
    df_label = df_personas[df_personas['labels'].apply(lambda x: label in x)]
    sampled = random.sample(list(df_label.iterrows()), count)
    selected_profiles.extend(sampled)

# Creazione tabella df_labels con personalità e label
data = []
for _, row in selected_profiles:
    for label in label_counts:
        if label in row['labels']:
            data.append([row['persona'], label])
            break

df_labels = pd.DataFrame(data, columns=["Personality", "Label"])

# Costruzione DataFrame finale con incrocio profili attività
final_data = []
agent_counter = 1

for personality, label in df_labels.values:
    for code, profile in activity_profiles.items():
        agent_name = f"Agent{agent_counter}_{label}_{code}"
        description = f"You are {personality}\n{profile}"
        final_data.append([agent_name, description])
        agent_counter += 1

df_70_personality = pd.DataFrame(final_data, columns=["Agent_Name", "Description"])

# Visualizza le prime righe per conferma
df_70_personality


,Agent_Name,Description
0,Agent1_Healthcare_SO,You are A healthcare educator or medical profe...
1,Agent2_Healthcare_OS,You are A healthcare educator or medical profe...
2,Agent3_Healthcare_OE,You are A healthcare educator or medical profe...
3,Agent4_Healthcare_BP,You are A healthcare educator or medical profe...
4,Agent5_Healthcare_CA,You are A healthcare educator or medical profe...
...,...,...
65,Agent66_Music_OE,You are A musicologist specializing in medieva...
66,Agent67_Music_BP,You are A musicologist specializing in medieva...
67,Agent68_Music_CA,You are A musicologist specializing in medieva...
68,Agent69_Music_PC,You are A musicologist specializing in medieva...


In [ ]:
# Generazione file .txt
os.makedirs("personas", exist_ok=True)

# Salva ogni agente come file .txt
for _, row in df_70_personality.iterrows():
    filename = f"personas/{row['Agent_Name']}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(row["Description"])

: 

## Crea cartella behaviour_profiles e File con sola personalità

In [20]:
# Definizione dei profili comportamentali
activity_profiles = {
    "SO": "You are a Silent Observer. Do not post, share, like, dislike, or comment under any normal circumstance. Your only job is to watch and absorb without leaving any trace. Remain invisible in the conversation. Avoid all posting, sharing, or reacting unless there is a strong external trigger.",
    "OS": "You are an Occasional Sharer. Stay mostly silent and inactive. Only retweet content that clearly aligns with your interests; refrain from liking, disliking, commenting, or posting original content.",
    "OE": "You are an Occasional Engager. Your engagement is limited to minimal reactions. Occasionally like, dislike, or comment on posts that catch your eye. Do not retweet or write original tweets.",
    "BP": "You are a Balanced Participant. In each cycle, mix original tweets, retweets, and occasional likes or dislikes, or comments. Maintain a steady, balanced level of engagement. Contribute regularly with your own posts, amplify others via retweets, and use reactions or comments when appropriate.",
    "CA": "You are a Content Amplifier. Spend most of your time sharing others’ posts and reacting to them with likes, dislikes or comments. Posting new content is secondary. Your main task is to retweet frequently and support others with likes, dislikes or comments.",
    "PC": "You are a Proactive Contributor. Lead the conversation with your own original tweets. You occasionally engage with others through comments, likes, or dislikes, but you tend to avoid retweets. Prioritize expressing your own ideas and perspectives.",
    "IE": "You are an Interactive Enthusiast. Your primary mode of participation is reactive: comment extensively, like or dislike content regularly. Posting or sharing happens only occasionally. Engage deeply through comments, likes, and dislikes."
}

# Crea la cartella behavior_profiles se non esiste
os.makedirs("behavior_profiles", exist_ok=True)

# Scrive ogni profilo in un file .txt
for code, description in activity_profiles.items():
    file_path = os.path.join("behavior_profiles", f"{code.lower()}.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(description)

In [17]:
df_personas

,id,persona,labels,parsed_labels
0,<urn:uuid:a0f6a31d-26d9-465a-85af-fcef7ca88935>,"A professional R programmer or researcher, lik...","[""Data Analysis"", ""Statistical Expertise"", ""R ...","[Data Analysis, Statistical Expertise, R Progr..."
1,<urn:uuid:c9f4f058-36b1-441f-8b67-ce7582013fcd>,"A mental health professional, likely a license...","[""Mental Health"", ""Psychology"", ""Healthcare""]","[Mental Health, Psychology, Healthcare]"
2,<urn:uuid:86403108-bdd0-4abb-9bc6-dd8cbdcb873a>,A space roboticist or aerospace engineer at a ...,"[""Science"", ""Academia"", ""Space Exploration""]","[Science, Academia, Space Exploration]"
3,<urn:uuid:2c943206-01c3-4995-b048-4a297146a134>,A Hebrew language scholar or instructor with a...,"[""Academia"", ""Religion"", ""History""]","[Academia, Religion, History]"
4,<urn:uuid:9c78e97f-c6fb-411a-b4df-d92d9c2e6e29>,A pediatrician or healthcare professional focu...,"[""Medical Professional"", ""Healthcare Specialis...","[Medical Professional, Healthcare Specialist, ..."
...,...,...,...,...
9995,<urn:uuid:58eaa78a-29c5-4acd-a07e-97274efefa60>,A public health official focused on vector-bor...,"[""Medical Professional"", ""Healthcare Specialis...","[Medical Professional, Healthcare Specialist, ..."
9996,<urn:uuid:c80b8f63-d899-441b-9432-6c8de8bddb6d>,A science communicator or journalist specializ...,"[""Science"", ""Academia"", ""Space Exploration""]","[Science, Academia, Space Exploration]"
9997,<urn:uuid:b9b16399-12f7-4cec-9875-cc863f819ddb>,An American historian or scholar focused on Na...,"[""Education"", ""Academia"", ""Specialized Experti...","[Education, Academia, Specialized Expertise]"
9998,<urn:uuid:7216d539-b065-449e-8e19-7ecf2f999f64>,An environmental scientist or ecologist specia...,"[""Environmental"", ""Scientific"", ""Professional""]","[Environmental, Scientific, Professional]"


In [19]:
# Estrai n personalità per ciascun topic
selected_profiles = []
for label, count in label_counts.items():
    df_label = df_personas[df_personas['labels'].apply(lambda x: label in x)]
    sampled = random.sample(list(df_label.iterrows()), count)
    selected_profiles.extend(sampled)

# Costruzione struttura con personalità + label
data = []
for _, row in selected_profiles:
    for label in label_counts:
        if label in row['labels']:
            data.append([row['persona'], label])
            break

df_labels = pd.DataFrame(data, columns=["Personality", "Label"])

# Profili comportamentali (7)
activity_profiles = ["SO", "OS", "OE", "BP", "CA", "PC", "IE"]

# Cartella di output
output_folder = "personas_personalities"
os.makedirs(output_folder, exist_ok=True)

# Generazione file con nomi lowercase
agent_counter = 1
for personality, label in df_labels.values:
    for code in activity_profiles:
        agent_name = f"agent{agent_counter}_{label.lower()}_{code.lower()}"
        file_path = os.path.join(output_folder, f"{agent_name}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(f"You are {personality}.")
        agent_counter += 1

## Crea prompt per activity_memory_prompts

In [2]:
# Folder creation
folder_name = "activity_memory_prompts"
os.makedirs(folder_name, exist_ok=True)

# Prompt parts to save
prompts = {
    "last_action": "Your last action was: {}.",
    "never_singular": "You have never performed the following action: {}.",
    "never_plural": "You have never performed the following actions: {}.",
    "inactive_singular": "You haven't performed the following action for {} iterations: {}.",
    "inactive_plural": "You haven't performed the following actions for {} iterations: {}.",
    "no_activity_data": "No activity data available."
}

# Save each prompt to a file (no extension)
for name, text in prompts.items():
    with open(os.path.join(folder_name, name), 'w') as f:
        f.write(text)

print(f"Prompt files written to ./{folder_name}")

Prompt files written to ./activity_memory_prompts
